# Beginning Project Dashboard

Oh boy here goes a bunch of stuff

In [154]:
# Common imports from Python standard libraries
from dataclasses import dataclass
import math

# Common imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import time
import datetime as dt
import dateutil as du

# Science imports
import scipy
from scipy import stats
from scipy import cluster
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity
import sklearn.metrics
from statistics import mean

# Useful imports for visualization
import seaborn
import networkx as nx
import pyvis as pv
import plotly as py
from plotly import subplots

# Plotly express and dash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff


# Constants
EPS = 0.0000001
FINAL = False

# Some useful functions
def almost_equal(a,b,eps=EPS):
    return abs(a-b) <= eps

def binsearch(func, vv = 0.025, x_l = -100, x_r = 100, eps=0.00001):
    '''
    Search for x such that func(x) == vv,
    within a range of +- eps,
    with x between x_l and x_r.
    '''
    v_l = func(x_l)
    v_r = func(x_r)
    
    x_m = (x_l + x_r)/2
    v_m = func(x_m)
    
    while abs(v_m - vv) >= eps:
        if v_m < vv:
            x_l = x_m
        elif v_m > vv:
            x_r = x_m
        else:
            print("weird, this shouldn't be possible")
            return x_m
            
        v_l = func(x_l)
        v_r = func(x_r)
        x_m = (x_l + x_r)/2
        v_m = func(x_m)
    
    return x_m

# Code provided from HW4
def stdev(X):
    m = mean(X)
    return math.sqrt(sum((x-m)**2 for x in X) / len(X))

def degreesOfFreedom(X, Y):
    s1 = (stdev(X)**2)
    s2 = (stdev(Y)**2)
    df = (s1 / len(X) + s2 / len(Y))**2 / ((s1 / len(X))**2 / (len(X) - 1) + (s2 / len(Y))**2 /
(len(Y) - 1))
    return(df)



In [3]:
df_full = pd.read_csv("../DATASETS/US_Accidents_Dec20_updated.csv")
df_full

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-2716600,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.10891,-83.09286,40.11206,-83.03187,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2716601,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.86542,-84.06280,39.86501,-84.04873,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-2716602,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10266,-84.52468,39.10209,-84.52396,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-2716603,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.10148,-84.52341,39.09841,-84.52241,0.219,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
4,A-2716604,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.06213,-81.53784,41.06217,-81.53547,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516059,A-4239402,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.00248,-117.37936,33.99888,-117.37094,0.543,At Market St - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day
1516060,A-4239403,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.76696,-117.14806,32.76555,-117.15363,0.338,At Camino Del Rio/Mission Center Rd - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day
1516061,A-4239404,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.77545,-117.84779,33.77740,-117.85727,0.561,At Glassell St/Grand Ave - Accident. in the ri...,...,False,False,False,False,False,False,Day,Day,Day,Day
1516062,A-4239405,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.99246,-118.40302,33.98311,-118.39565,0.772,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


In [199]:
df = df_full.sample(n = 3000, ignore_index = True)

# 2. Start up dashboard.

Oh man.

Need to do:

- Clustering
    - Hierarchical
    - K-Means
- Gaussian Mixture Model (classification)
- Network Analysis (visualization)
- Correlation Analysis:
    - Linear regression
    - Pearson correlation
    - Kernel Density Estimate
- Hypothesis testing
    - t-test
    - p-value
- Statistics
    - Boxplots
    - Violin plots
    - Histogram
- Geospatial Analysis
    - Cartogram map
- NLP / text mining:
    - Word cloud
    - Bar plot
- Basics:
    - Line graph


---
---

## (1) Boxplots, Violin Plots

Let's use these to understand the **distance** of the accidents, and how it relates to these keys:

1. 'Stop'
2. 'Traffic_Calming'
3. 'Roundabout'
4. 'Traffic_Signal'


In [6]:
df.keys()

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [8]:
dist_checklist_keys = ['Stop', 'Traffic_Calming', 'Roundabout', 'Traffic_Signal', 'Crossing', 'Railway', 'Bump']
distance_by_severity = {severity : np.array(df[df['Severity'] == severity]['Distance(mi)']) for severity in set(df['Severity'])}


app = dash.Dash(name="Distance by different traffic conditions")

app.layout = html.Div(
    [
        html.Div(
            [
                html.H2("Do traffic controls mitigate the impact of accidents?"),
                html.P('''
A serious traffic accident can leave one or more lanes of a road unusable.
After an accident, we measure the *distance* of affected road in miles.
Use this tool and explore how the severity of accidents relate to the presence of
traffic controls such as stop signs, traffic signals, roundabouts, traffic crossings,
train railways, speed-bumps, and 'traffic-calming' techniques
                '''),
                dcc.Checklist(
                    id = "severity-checklist",
                    options = [{'value' : str(x), 'label' : f'{str(x)}'}
                               for x in dist_checklist_keys],
                               #for x in [1, 2, 3, 4]],
                    value   = [],
                    labelStyle = {'display' : 'inline-block'}
                ),
            ]
        ),
        dcc.Graph(id = "severity-distance-boxplot")
    ]
)


@app.callback(
    Output("severity-distance-boxplot", "figure"), 
    [Input("severity-checklist", "value")]
)
def generate_plot(x):
    fig = px.violin(
        df,
        x = x,
        y = 'Distance(mi)',
        box = True,
        points='all'
    )
    return fig


app.run_server(port = 10001, debug=False)

Dash is running on http://127.0.0.1:10001/

 * Serving Flask app 'Distance by different traffic conditions' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:10001/ (Press CTRL+C to quit)


---
---

## (2) Histogram and KDE estimation

- Clustering
    - Hierarchical
    - K-Means
- Gaussian Mixture Model (classification)
- Network Analysis (visualization)
- Correlation Analysis:
    - Linear regression
    - Pearson correlation
    - Kernel Density Estimate
- Hypothesis testing
    - t-test
    - p-value
- Statistics
    - ~~Boxplots~~
    - ~~Violin plots~~
    - Histogram
- Geospatial Analysis
    - Cartogram map
- NLP / text mining:
    - Word cloud
    - Bar plot
- Basics:
    - Line graph


Let's do **KDE and Histogram** now.

How present are the following weather conditions in traffic accidents?



In [38]:
weather_keys = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']

app = dash.Dash(name="Weather conditions and traffic accidents")

app.layout = html.Div(
    [
        html.Div(
            [
                html.H2("abc"),
                html.P('''
Explore the distribution of different weather conditions during traffic accidents.
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "weather-kde"),
                dcc.Checklist(
                    id = "weather-checklist",
                    options = [{'value' : str(x), 'label' : f'{str(x)}'}
                               for x in weather_keys],
                               #for x in [1, 2, 3, 4]],
                    value   = ["Temperature(F)"],
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.Slider(
                    id="weather-binsize",
                    min=0.1,
                    max=10.1,
                    value=1.0, 
                    marks = {x : str(x) for x in [0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0]}
                )
            ]
        )
                
    ]
)


@app.callback(
    Output("weather-kde", "figure"), 
    [Input("weather-checklist", "value"),
     Input("weather-binsize", "value")
    ]
)
def generate_plot(labels, binsize):
    '''
    fig = px.histogram(
        df,
        x = x,
        #marginal = 'violin',
        histnorm = 'probability density',
        nbins = bins
    )
    return fig
    '''
    data = np.nan_to_num(np.array(df[labels]), copy=True, nan=0.0, posinf=0.0, neginf=0.0).T
    fig = ff.create_distplot(data, labels, bin_size=binsize)
    return fig


app.run_server(port = 10001, debug=False)

Dash is running on http://127.0.0.1:10001/

Dash is running on http://127.0.0.1:10001/

Dash is running on http://127.0.0.1:10001/

Dash is running on http://127.0.0.1:10001/

Dash is running on http://127.0.0.1:10001/

Dash is running on http://127.0.0.1:10001/

Dash is running on http://127.0.0.1:10001/

 * Serving Flask app '1234' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:10001/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2021 13:39:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:40] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:40] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:40] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:40] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 13:39:48] "POST /_dash-update-component HTTP/1.1" 2

---
---


## (3) Hypothesis Testing

- Clustering
    - Hierarchical
    - K-Means
- Gaussian Mixture Model (classification)
- Network Analysis (visualization)
- Correlation Analysis:
    - Linear regression
    - Pearson correlation
    - ~~Kernel Density Estimate~~
- Hypothesis testing
    - t-test
    - p-value
- ~~Statistics~~
    - ~~Boxplots~~
    - ~~Violin plots~~
    - ~~Histogram~~
- Geospatial Analysis
    - Cartogram map
- NLP / text mining:
    - Word cloud
    - Bar plot
- Basics:
    - Line graph


Let's do a t-test against this null hypothesis: **The presence of stop signs have no impact on the distance of an accident.**

In [198]:
test_keys = ['Stop', 'Crossing', 'Traffic_Calming', 'Traffic_Signal']
test_metrics = ['Distance(mi)', 'Severity']

app = dash.Dash(name="Hypothesis testing")

app.layout = html.Div(
    [
        html.Div(
            [
                html.H2("Hypothesis testing"),
                html.P('''
Do various traffic conditions impact how bad accidents get?
Explore combinations of T-tests here and their respective p-scores.
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "hypothesis-graph"),
                dcc.RadioItems(
                    id = "hypothesis-keys",
                    options = [{'value' : x, 'label' : f"Test how '{x}' ..."}
                               for x in test_keys],
                    value = ["Stop"],
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.RadioItems(
                    id = "hypothesis-metric",
                    options = [{'value' : x, 'label' : f"... impacts metric '{x}'"}
                               for x in metrics],
                    value = ["Distance(mi)"],
                    labelStyle = {'display' : 'inline-block'}
                )
            ]
        )
                
    ]
)

@app.callback(
    Output("hypothesis-graph", "figure"), 
    [Input("hypothesis-keys", "value"),
     Input("hypothesis-metric", "value")
    ]
)
def get_figure(key = 'Stop', metric = 'Distance(mi)', show_boxes=False):
    # From our dataframe, get items with and without boolean key
    key_with = df[df[key] == True][metric]
    key_without = df[df[key] == False][metric]

    # get degrees of freedom
    v_obs = degreesOfFreedom(key_with, key_without)
    # Get the t-pdf of our two values
    pdf_with = lambda x: stats.t.pdf(
        x, df = v_obs, loc = key_with.mean(), scale = key_with.std()
    )
    pdf_without = lambda x: stats.t.pdf(
        x, df = v_obs, loc = key_without.mean(), scale = key_without.std()
    )
    # get statistics
    t_obs, p_obs = scipy.stats.ttest_ind(dist_at_stop, dist_no_stop)

    # Lines for graphing
    Xs = np.linspace(-6, 6, 1000)
    Ys_with = [pdf_with(x) for x in Xs]
    Ys_without = [pdf_without(x) for x in Xs]

    # Create figures
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=Xs, y=Ys_with, mode='lines'))
    fig.add_trace(go.Scatter(x=Xs, y=Ys_without, mode='lines'))
    fig.add_trace(
        go.Scatter(
            x = [key_with.mean()],
            y = [max(Ys_with) + .20],
            text = [f"T-stat: {t_obs:.3f}    P-score: {p_obs:.3f}"],
            mode = "text"
        )
    )
    
    if show_boxes:
        fig.add_trace(go.Box(x=key_with))
        fig.add_trace(go.Box(x=key_without))
    
    return fig


app.run_server(port = 10001, debug=False)

Dash is running on http://127.0.0.1:10001/

Dash is running on http://127.0.0.1:10001/

 * Serving Flask app 'Hypothesis testing' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:10001/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2021 19:30:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:30:03] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:30:03] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:30:03] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:30:03] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:30:03] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/_callback.py", line 151, 

127.0.0.1 - - [10/Dec/2021 19:30:03] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/_callback.py", line 151, 

127.0.0.1 - - [10/Dec/2021 19:30:15] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [10/Dec/2021 19:30:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:30:17] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/_callback.py", line 151, 

127.0.0.1 - - [10/Dec/2021 19:30:17] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [10/Dec/2021 19:30:19] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/home/lynn/miniconda3/envs/5520/lib/python3.9/site-packages/dash/_callback.py", line 151, 

127.0.0.1 - - [10/Dec/2021 19:30:21] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [10/Dec/2021 19:30:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:30:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:30:25] "POST /_dash-update-component HTTP/1.1" 200 -


---

---


## (4) Cartogram map

- Clustering
    - Hierarchical
    - K-Means
- Gaussian Mixture Model (classification)
- Network Analysis (visualization)
- Correlation Analysis:
    - Linear regression
    - Pearson correlation
    - ~~Kernel Density Estimate~~
- ~~Hypothesis testing~~
    - ~~t-test~~
    - ~~p-value~~
- ~~Statistics~~
    - ~~Boxplots~~
    - ~~Violin plots~~
    - ~~Histogram~~
- Geospatial Analysis
    - Cartogram map
- NLP / text mining:
    - Word cloud
    - Bar plot
- Basics:
    - Line graph

This one is simple: Let's plot all the accidents by lat-lon, along with state shapes. Let users explore.

We might even add a textbox, letting users:

1. Select States or Zipcodes
2. Select dates
3. etc.

The next item, we'll do K-means clustering and label a map that way.

So...

1. US state shapefiles
2. Plot those
3. Scatterplot of accidents (latlon)
4. Select by state or zipcode


In [203]:
df.keys()

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [261]:
['Any'] + dist_checklist_keys

['Any',
 'Stop',
 'Traffic_Calming',
 'Roundabout',
 'Traffic_Signal',
 'Crossing',
 'Railway',
 'Bump']

In [ ]:
app = dash.Dash(name="Map")

app.layout = html.Div(
    [
        html.Div(
            [
                html.H2("Map"),
                html.P('''
The dataset labels accidents by the severity of the imapct it has on traffic,
on a scale of 1 (not severe) to 4 (most severe). Explore the distribution of
severity of these accidents here, and how they relate with different traffic measures.
                '''),
            ]
        ),
        html.Div(
            [
                dcc.Graph(id = "accidents-map"),
                dcc.RadioItems(
                    id = "map-severity",
                    options = [{'value' : x, 'label' : f"Severity {x}"}
                               for x in [1,2,3,4]],
                    value = 1,
                    labelStyle = {'display' : 'inline-block'}
                ),
                dcc.RadioItems(
                    id = "severity-checklist-map",
                    options = [{'value' : str(x), 'label' : f'{str(x)}'}
                               for x in ['Any'] + dist_checklist_keys],
                               #for x in [1, 2, 3, 4]],
                    value   = 'Any',
                    labelStyle = {'display' : 'inline-block'}
                ),
            ]
        )
                
    ]
)

@app.callback(
    Output("accidents-map", "figure"), 
    [Input("map-severity", "value"),
     Input("severity-checklist-map", "value"),
    ]
)
def get_figure(severity = [1], traffic_measure = 'Any'):
    colors = [
        'rgba(0,0,0,0)',
        'rgba(0,128,192,192)',
        'rgba(128,0,192,192)',
        'rgba(256,0,128,192)', 
        'rgba(256,128,0,192)'
    ]
    
    if traffic_measure == 'Any':
        df_subset = df[df['Severity'] == severity]
    else:
        df_subset = df[df['Severity'] == severity][df[traffic_measure] == True]
    
    color_str = colors[severity]
    color = [color_str for _ in range(len(df_subset))]
    
    fig = go.Figure(
        data = go.Scattergeo(
            lat=df_subset['Start_Lat'],
            lon=df_subset['Start_Lng'],
            marker_color = color,
            marker_size = 4,
        )
    )
    
    fig.update_layout(geo_scope='usa')
    
    return fig

    
app.run_server(port = 10001, debug=False)

With this map, we have (weakly) cartogram and network analysis.

I think the remaining steps are as follows:

1. Repeat this map with clustering
2. (Maybe show hierarchy analysis?)
3. Correlation magic modal:
    - User selects any X and Y keys
    - Plots scatter, regression fit line, with pearson correlation coefficient

And of course, skip word cloud / bar plot.

---
---



## (5) Clustering

- Clustering
    - Hierarchical
    - K-Means
- Gaussian Mixture Model (classification)
- ~~Network Analysis (visualization)~~
- Correlation Analysis:
    - Linear regression
    - Pearson correlation
    - ~~Kernel Density Estimate~~
- ~~Hypothesis testing~~
    - ~~t-test~~
    - ~~p-value~~
- ~~Statistics~~
    - ~~Boxplots~~
    - ~~Violin plots~~
    - ~~Histogram~~
- ~~Geospatial Analysis~~
    - ~~Cartogram map~~
- NLP / text mining:
    - Word cloud
    - Bar plot
- Basics:
    - Line graph